In [1]:
from context import *

# Quantum Device

## ClassicalShadow

`ClassicalShadow(state, circuit)` is a high-level interface to simulate the classical shadow data acquisition protocol on a stabilizer state $\rho$ via a random Clifford circuit $U$.

**Parameters:**
* `state` - a `StabilizerState` representing the base state $\rho$.
* `circuit` - a `CliffordCircuit` representing the unitary channel $U$ in the measurement protocol.

### Methods

#### snapshots

`ClassicalShadow.snapshots(nsample)` provides a generator to serve $n_\text{sample}$ of classical snapshot states from the posterior POVM $\mathcal{E}_{\sigma|\rho}$.

* Forward protocol (physical):
    * Get a copy of $\rho$.
    * Evolve the state by the unitary circuit $\rho\to \rho'=U\rho U^\dagger$.
    * Measure in the computational basis $\rho'\to|b\rangle\langle b|$.
    * Collect the snapshot state $\hat{\sigma}=U^\dagger |b\rangle\langle b| U$.
    
* Backward protocol (computational):
    * Start from the all-up state $M_0=|\vec{0}\rangle\langle \vec{0}|=\prod_i\frac{1+Z_i}{2}$.
    * Transform backward through the unitary channel $M_0\to M= U^\dagger M_0 U=\prod_i\frac{1+Z'_i}{2}$ to the measurement basis.
    * Get a copy of $\rho$, measure observables $Z'_i$ , obtain measurement outcome $b_i$.
    * Collect the post measurement state $\hat{\sigma}=\prod_{i}\frac{1+(-)^{b_i}Z'_i}{2}=\prod_{i}\frac{1+g_i}{2}$ (by redefining $g_i=(-)^{b_i}Z'_i$).
    
Both protocols collect the classical snapshot $\hat{\sigma}$ correctly. The backward protocol is implement in the computation.

Example:

In [3]:
shd = qst.ClassicalShadow(qst.ghz_state(3), qst.global_rcc(3))
shd

ClassicalShadow(
  StabilizerState(
     +ZZI
     +IZZ
     +XXX),
  CliffordCircuit(
    |[0,1,2]|))

In [4]:
for snapshot in shd.snapshots(5):
    print(snapshot)

StabilizerState(
   +ZYI
   -YXZ
   +XXX)
StabilizerState(
   +ZZI
   +XXZ
   -XYY)
StabilizerState(
   +ZYI
   -YXI
   -YXY)
StabilizerState(
   -ZYX
   +YYZ
   +IZY)
StabilizerState(
   +ZZI
   -YYZ
   -IZY)


### Fidelity Estimate

Estimate the fidelity of the original state and the reconstructed state under classical shadow tomography.

* Global Clifford measurement,
$$\mathrm{Tr}(\rho\tilde{\rho})=\mathbb{E}_{\mathcal{E}_{\sigma|\rho}}\mathrm{Tr}(\rho\mathcal{M}^{-1}[\sigma]) = (2^N+1)\mathbb{E}_{\mathcal{E}_{\sigma|\rho}}\mathrm{Tr}(\rho\sigma)-1.$$

In [5]:
N = 50
nsample = 1000
rho = qst.ghz_state(N)
circ = qst.global_rcc(N)
shd = qst.ClassicalShadow(rho, circ)
acc = 0.
for sigma in shd.snapshots(nsample):
    acc += rho.expect(sigma)
F = (2**rho.N + 1)*(acc/nsample) - 1.
print('fidelity: {:8.6f}'.format(F))

fidelity: 0.991000


* Local Clifford measurement,
$$\mathrm{Tr}(\rho\tilde{\rho})=\mathbb{E}_{\mathcal{E}_{\sigma|\rho}}\mathrm{Tr}(\rho\mathcal{M}^{-1}[\sigma]) = \mathbb{E}_{\mathcal{E}_{\sigma|\rho}}\mathrm{Tr}\left(\rho\prod_i(3\sigma_i-\mathbb{1}_i)\right) = \mathbb{E}_{\mathcal{E}_{\sigma|\rho}}\mathrm{Tr}\left(\rho\prod_i\frac{3g_i+\mathbb{1}}{2}\right).$$

In [6]:
N = 5
nsample = 1000
rho = qst.ghz_state(N)
circ = qst.onsite_rcc(N)
shd = qst.ClassicalShadow(rho, circ)
acc = 0.
for sigma in shd.snapshots(nsample):
    acc += rho.expect(sigma.density_matrix, z=3)
F = acc/nsample
print('fidelity: {:8.6f}'.format(F.real))

fidelity: 1.040938
